<a href="https://colab.research.google.com/github/Divya-demo1/Deep-Learning-Projects/blob/master/Distracted_driver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
ImageDataGenerator.flow_from_directory?

In [3]:
 !wget  https://www.dropbox.com/s/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip?dl=0

--2020-02-14 18:17:47--  https://www.dropbox.com/s/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip [following]
--2020-02-14 18:17:47--  https://www.dropbox.com/s/raw/0vyzjcqsdl6cqi2/state-farm-distracted-driver-detection.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1852a207450ace991b52885ab3.dl.dropboxusercontent.com/cd/0/inline/AyHnLNUQcTOadekOWuOEUJwVFcNsphuweWfYIz3JiiWJb4MLt-kSqlWlZwaNUkM5esrjrdVoT0Q_I9EpmsvmI-38_f65uT8c0iOP_6lXhiDR9w/file# [following]
--2020-02-14 18:17:47--  https://uc1852a207450ace991b52885ab3.dl.dropboxusercontent.com/cd/0/inline/AyHnLNUQcTOadekOWuOEUJwVFcNsphuweWfYIz3J

In [4]:
 !unzip state-farm-distracted-driver-detection.zip?dl=0

Archive:  state-farm-distracted-driver-detection.zip?dl=0
  inflating: sample_submission.csv   
  inflating: driver_imgs_list.csv    
  inflating: imgs.zip                


In [5]:
 !unzip imgs.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/c7/img_56699.jpg  
  inflating: train/c7/img_56717.jpg  
  inflating: train/c7/img_56724.jpg  
  inflating: train/c7/img_56768.jpg  
  inflating: train/c7/img_56779.jpg  
  inflating: train/c7/img_56849.jpg  
  inflating: train/c7/img_56899.jpg  
  inflating: train/c7/img_56938.jpg  
  inflating: train/c7/img_5706.jpg   
  inflating: train/c7/img_57076.jpg  
  inflating: train/c7/img_57122.jpg  
  inflating: train/c7/img_57137.jpg  
  inflating: train/c7/img_5715.jpg   
  inflating: train/c7/img_57278.jpg  
  inflating: train/c7/img_57321.jpg  
  inflating: train/c7/img_57345.jpg  
  inflating: train/c7/img_57355.jpg  
  inflating: train/c7/img_5741.jpg   
  inflating: train/c7/img_57433.jpg  
  inflating: train/c7/img_5753.jpg   
  inflating: train/c7/img_57559.jpg  
  inflating: train/c7/img_57594.jpg  
  inflating: train/c7/img_5771.jpg   
  inflating: train/c7/img_57712.jpg  
  inflating: train/c7/img_57733.jpg  

In [6]:
os.listdir()

['.config',
 'sample_submission.csv',
 'test',
 'state-farm-distracted-driver-detection.zip?dl=0',
 'driver_imgs_list.csv',
 'imgs.zip',
 'train',
 'sample_data']

In [7]:
df = pd.read_csv('driver_imgs_list.csv')
df.head(5)

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [0]:
import pandas as pd
import numpy as np
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.layers import Dense,GlobalAveragePooling2D,Dropout
from keras.models import Model

In [9]:
#TRANSFER LEARNING
#using a pretrained model
base_model= VGG16(weights='imagenet',include_top=False)





58892288/58889256 [==============================] - 1s 0us/step








In [10]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [11]:
CLASSES = 10 # images are of 10 classes
x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x=(Dense(256,activation='relu'))(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
for layer in base_model.layers:
    layer.trainable = False
      
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0   

In [14]:
os.listdir()

['.config',
 'sample_submission.csv',
 'test',
 'state-farm-distracted-driver-detection.zip?dl=0',
 'driver_imgs_list.csv',
 'imgs.zip',
 'train',
 'sample_data']

In [15]:
os.listdir('train')

['c6', 'c4', 'c0', 'c1', 'c9', 'c3', 'c5', 'c2', 'c8', 'c7']

In [16]:
os.listdir('test')

['img_68846.jpg',
 'img_32522.jpg',
 'img_52846.jpg',
 'img_64011.jpg',
 'img_66838.jpg',
 'img_51049.jpg',
 'img_96865.jpg',
 'img_89137.jpg',
 'img_84550.jpg',
 'img_51953.jpg',
 'img_96487.jpg',
 'img_100512.jpg',
 'img_73508.jpg',
 'img_33464.jpg',
 'img_100887.jpg',
 'img_99598.jpg',
 'img_7066.jpg',
 'img_27817.jpg',
 'img_55423.jpg',
 'img_9607.jpg',
 'img_1427.jpg',
 'img_28132.jpg',
 'img_60995.jpg',
 'img_28980.jpg',
 'img_91250.jpg',
 'img_48139.jpg',
 'img_95548.jpg',
 'img_6494.jpg',
 'img_83215.jpg',
 'img_28266.jpg',
 'img_66596.jpg',
 'img_44545.jpg',
 'img_53472.jpg',
 'img_72466.jpg',
 'img_66895.jpg',
 'img_53801.jpg',
 'img_36478.jpg',
 'img_3503.jpg',
 'img_38515.jpg',
 'img_79612.jpg',
 'img_40430.jpg',
 'img_34663.jpg',
 'img_1561.jpg',
 'img_86801.jpg',
 'img_59902.jpg',
 'img_4179.jpg',
 'img_27238.jpg',
 'img_66931.jpg',
 'img_92693.jpg',
 'img_89090.jpg',
 'img_51527.jpg',
 'img_63898.jpg',
 'img_24855.jpg',
 'img_68257.jpg',
 'img_92012.jpg',
 'img_25743.jpg

In [17]:
WIDTH = 299
HEIGHT = 299
BATCH_SIZE = 32
TRAIN_DIR=r'train'
TEST_DIR=r'test'

#data augumentation 
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
#data loading in batches
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')
    
validation_generator = validation_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 22424 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


In [18]:
EPOCHS = 30
BATCH_SIZE = 32
STEPS_PER_EPOCH = 320
VALIDATION_STEPS=64
model.fit_generator(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
320/320 [==============================] - 257s 804ms/step - loss: 2.2649 - acc: 0.2515
Epoch 2/30
320/320 [==============================] - 247s 771ms/step - loss: 1.5710 - acc: 0.4514
Epoch 3/30
320/320 [==============================] - 245s 766ms/step - loss: 1.3398 - acc: 0.5360
Epoch 4/30
320/320 [==============================] - 251s 785ms/step - loss: 1.1684 - acc: 0.5999
Epoch 5/30
320/320 [==============================] - 254s 793ms/step - loss: 1.0796 - acc: 0.6264
Epoch 6/30
320/320 [==============================] - 246s 770ms/step - loss: 1.0253 - acc: 0.6509
Epoch 7/30
320/320 [==============================] - 245s 767ms/step - loss: 0.9833 - acc: 0.6685
Epoch 8/30
320/320 [==============================] - 247s 771ms/step - loss: 0.9338 - acc: 0.6807
Epoch 9/30
320/320 [==============================] - 252s 788ms/step - loss: 0.9023 - acc: 0.6966
Epoch 10/30
3

In [0]:
from tensorflow.python.keras.preprocessing import image

In [0]:
img = image.load_img(r'test/img_50360.jpg', target_size=(299,299))

In [0]:
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)
predictions = model.predict(x)[0]

In [22]:
predictions

array([2.1857284e-01, 7.2250557e-01, 7.3354036e-05, 4.9901393e-04,
       8.6840619e-06, 4.0240064e-07, 5.3737309e-02, 2.2621821e-06,
       4.4543738e-03, 1.4624355e-04], dtype=float32)

In [23]:
train_generator.class_indices

{'c0': 0,
 'c1': 1,
 'c2': 2,
 'c3': 3,
 'c4': 4,
 'c5': 5,
 'c6': 6,
 'c7': 7,
 'c8': 8,
 'c9': 9}

In [25]:
np.argmax(predictions) 
#at 1th position c1 picture is texting right

1

In [0]:
# Saving the model 

In [0]:
from tensorflow.keras.models import model_from_json

In [0]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


In [0]:
model.save_weights("model.h5")

In [0]:
# loading the model 

In [31]:
from tensorflow.keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
____________________________________________

In [0]:
loaded_model.load_weights("model.h5")

In [0]:
# make prediction with saved model

In [0]:
predictions = loaded_model.predict(x)[0]

In [35]:
predictions


array([2.1857284e-01, 7.2250557e-01, 7.3354036e-05, 4.9901393e-04,
       8.6840619e-06, 4.0240064e-07, 5.3737309e-02, 2.2621821e-06,
       4.4543738e-03, 1.4624355e-04], dtype=float32)